In [2]:
import numpy as np 
import math
from collections import defaultdict

In [3]:
M = 2048
N = 2048
K = 2048
BK = 128
BM = 128 
BN = 128
num_sms = 170
GM = M//BM
GN = N//BN
GK = K//BK 
num_tiles = GM*GN 

sm_counts = [0]*170
n_tiles_hit = 0
tile_set = set()
sm_map = defaultdict(list)



def tile_for(sm, step, num_sms, num_tiles): 
    val = (sm + step * num_sms)
    if val >= num_tiles: 
      return None
    else:
      return val

def tile_id_C_tile_coord(tile_id, GM,GN):
  return (tile_id/GN, tile_id%GN)



for step in range(math.ceil(num_tiles/num_sms)):
  for s in range(num_sms): 
    tile_id = tile_for(s,step,num_sms,num_tiles)
    if tile_id != None:
      if tile_id not in tile_set:
        tile_set.add(tile_id)
        sm_counts[s] += 1
        sm_map[s].append(tile_id)
      else:
        print("oopsie, two different sms or two different iterations hitting the same tile")
        break
    
  
  
sm_map

defaultdict(list,
            {0: [0, 170],
             1: [1, 171],
             2: [2, 172],
             3: [3, 173],
             4: [4, 174],
             5: [5, 175],
             6: [6, 176],
             7: [7, 177],
             8: [8, 178],
             9: [9, 179],
             10: [10, 180],
             11: [11, 181],
             12: [12, 182],
             13: [13, 183],
             14: [14, 184],
             15: [15, 185],
             16: [16, 186],
             17: [17, 187],
             18: [18, 188],
             19: [19, 189],
             20: [20, 190],
             21: [21, 191],
             22: [22, 192],
             23: [23, 193],
             24: [24, 194],
             25: [25, 195],
             26: [26, 196],
             27: [27, 197],
             28: [28, 198],
             29: [29, 199],
             30: [30, 200],
             31: [31, 201],
             32: [32, 202],
             33: [33, 203],
             34: [34, 204],
             35: [35, 

In [4]:
import math
from collections import defaultdict
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def generate_schedule_viz(filename="sm_mapping_viz.html"):
    # Matrix and Grid Geometry
    M, N, K = 2048, 2048, 2048
    BM, BN, BK = 128, 128, 128
    num_sms = 170

    GM = M // BM
    GN = N // BN
    GK = K // BK
    num_tiles = GM * GN

    # 1. The Coordinate Function (Isolated for pure Form Extraction)
    def tile_id_to_coord(tile_id, GM, GN):
        return (int(tile_id // GN), int(tile_id % GN))

    # 2. Extract the Form: Map SMs to logical tiles
    sm_map = defaultdict(list)
    for step in range(math.ceil(num_tiles / num_sms)):
        for s in range(num_sms):
            tile_id = s + step * num_sms
            if tile_id < num_tiles:
                sm_map[s].append(tile_id)

    # 3. Initialize Text Grids for A, B, and C
    # We use strings to handle multiple SMs landing in the same visual cell
    A_text = [["" for _ in range(GK)] for _ in range(GM)]
    B_text = [["" for _ in range(GN)] for _ in range(GK)]
    C_text = [["" for _ in range(GN)] for _ in range(GM)]

    # We also track a "heat" value to color the cells based on SM density
    A_z = [[0 for _ in range(GK)] for _ in range(GM)]
    B_z = [[0 for _ in range(GN)] for _ in range(GK)]
    C_z = [[0 for _ in range(GN)] for _ in range(GM)]

    # 4. Project SMs onto the operands
    for sm, tiles in sm_map.items():
        for tile_id in tiles:
            m, n = tile_id_to_coord(tile_id, GM, GN)
            
            # C gets the exact SM
            C_text[m][n] += f"{sm} "
            C_z[m][n] += 1
            
            # A and B scatter the SM across the required row/col using modulo
            # This prevents all 16 SMs from stacking in the [m, 0] cell
            k_a = sm % GK
            k_b = sm % GK
            
            A_text[m][k_a] += f"{sm} "
            A_z[m][k_a] += 1
            
            B_text[k_b][n] += f"{sm} "
            B_z[k_b][n] += 1

    # 5. Build the Interactive Plotly Figure
    fig = make_subplots(
        rows=1, cols=3, 
        subplot_titles=(f"Operand A ({GM}x{GK})", f"Operand B ({GK}x{GN})", f"Output C ({GM}x{GN})"),
        horizontal_spacing=0.05
    )

    # Helper to add heatmaps
    def add_matrix_trace(fig, z_data, text_data, row, col, colorscale):
        fig.add_trace(
            go.Heatmap(
                z=z_data,
                text=text_data,
                texttemplate="%{text}",
                textfont={"size": 10, "color": "white"},
                colorscale=colorscale,
                showscale=False,
                hoverinfo="text"
            ),
            row=row, col=col
        )

    # Add traces with distinct brutalist color palettes
    add_matrix_trace(fig, A_z, A_text, 1, 1, "Teal")
    add_matrix_trace(fig, B_z, B_text, 1, 2, "Purp")
    add_matrix_trace(fig, C_z, C_text, 1, 3, "Inferno")

    # Update layout for structural beauty (invert Y axes to match matrix indexing)
    fig.update_layout(
        title_text="Operadic Type Form: SM Schedule Projection",
        title_x=0.5,
        plot_bgcolor="#0d1117",
        paper_bgcolor="#0d1117",
        font=dict(color="#c9d1d9", family="monospace"),
        height=700,
        margin=dict(t=80, b=40, l=40, r=40)
    )

    fig.update_yaxes(autorange="reversed", showgrid=True, gridcolor="#30363d")
    fig.update_xaxes(showgrid=True, gridcolor="#30363d")

    # Save to HTML
    fig.write_html(filename)
    print(f"Form successfully extracted. Open {filename} in your browser.")

if __name__ == "__main__":
    generate_schedule_viz()

Form successfully extracted. Open sm_mapping_viz.html in your browser.
